<a href="https://colab.research.google.com/github/ManuelBagasina/solarspell/blob/main/mySQL_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Only "Input Main Information" and "Upload files" needs to be changed. Run all chunks when they are changed
If you want to see the table, go to the 2nd section and run the last chunk

# Uploading to mySQL server

## Install Libraries

Uncomment the 2nd line "pip install" if it's your first time running this file

In [ ]:
# STEP 0: Install and Import Packages#
!pip install -q pandas sqlalchemy mysql-connector-python openpyxl #uncomment this if you need to install

import pandas as pd
from sqlalchemy import create_engine, Table, Column, MetaData, PrimaryKeyConstraint
from sqlalchemy.types import String, BigInteger, Float
from google.colab import files
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 14.5 MB/s eta 0:00:00


## Input Main information

Change this part if the server changes.
Change table name so that it's easier to find your dataset when you connect to Tableau

In [ ]:
# STEP 1: Server Config (EDIT THIS IF YOU HAVE TO)
user = "avnadmin"
password = "temporay_block_while_on_github"
host = "mysql-a9d31ef-asu-8f9a.h.aivencloud.com"
port = 16754
database = "defaultdb"

# Set the table name to whatever the table name should be. best if you put in the same file name
table_name = "lesotho"  # ✅ Change this per upload

# Clean up table_name: replace spaces, dots, and other symbols with underscores
table_name = re.sub(r'[^a-zA-Z0-9_]', '_', table_name)
table_name = table_name.lower()
print(f"Cleaned table name: {table_name}")

Cleaned table name: shortresptranslatedb2r_rwanda


## Upload Files (Please upload the CSV file and the pem file here)

Upload the csv file first.
Then upload the ca.pem file

In [ ]:
# STEP 2: Upload Files
print("📁 Step 1: Upload the CSV file (exported from Excel)")
uploaded_excel = files.upload()
excel_filename = list(uploaded_excel.keys())[0]

print("\n📁 Step 2: Upload the Aiven SSL Certificate (ca.pem)")
uploaded_cert = files.upload()
ssl_cert_filename = list(uploaded_cert.keys())[0]

📁 Step 1: Upload the CSV file (exported from Excel)


Saving Lesotho2024-07CombinedUsageData.xlsx - Combined Data.csv to Lesotho2024-07CombinedUsageData.xlsx - Combined Data.csv


'\nprint("\n📁 Step 2: Upload the Aiven SSL Certificate (ca.pem)")\nuploaded_cert = files.upload()\nssl_cert_filename = list(uploaded_cert.keys())[0]'

Run this and make sure the rows match the csv

In [ ]:
# STEP 3: Load Data
print(f"\n📊 Reading '{excel_filename}'...")
df = pd.read_csv(excel_filename)
print(f"✅ Loaded {df.shape[0]} rows × {df.shape[1]} columns.")


📊 Reading 'Lesotho2024-07CombinedUsageData.xlsx - Combined Data.csv'...
✅ Loaded 14548 rows × 20 columns.


## Under the Hood (Don't change)

Run all of this and please dont change anything. The data should be uploaded to the server here

In [ ]:
# STEP 4: Type Inference for SQL Columns
def infer_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return BigInteger()
    elif pd.api.types.is_float_dtype(dtype):
        return Float()
    elif pd.api.types.is_datetime64_any_dtype(dtype): # Add check for datetime types
        return String(255)  # Use String type for dates
    else:
        return String(255)  # Default to VARCHAR(255)

# Rename 'id' if it exists to avoid conflict
if "id" in df.columns:
    df.rename(columns={"id": "original_id"}, inplace=True)

# Insert new primary key at position 0
df.insert(0, "id", range(1, len(df) + 1))

# drop original id
if "original_id" in df.columns:
  df = df.drop("original_id", axis=1)
  print("Dropped 'original_id' column.")
else:
  print("'original_id' column not found.")

display(df.head())

Dropped 'original_id' column.


,id,title,activity_type,activity_date,browser,device_type,device_os,language,content_type,subject,parent_folder,referrer,keyword,rights_holder,location,audience,format,country,year_units_retrieved,date_retrieved
0,1,test,search,Mon Dec 18 2023 11:12:46 GMT-0700 (Mountain St...,Chrome Generic,Desktop,MacOSX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KoaliACL,NaN,NaN,Lesotho,2024,2024 -- 07
1,2,Core Subjects and Your Career,access_content,Mon Dec 18 2023 11:18:08 GMT-0700 (Mountain St...,Chrome Generic,Desktop,MacOSX,English,Guide,Social Studies,Careers and School,folder,NaN,NaN,KoaliACL,NaN,NaN,Lesotho,2024,2024 -- 07
2,3,A Useful Fly Trap,access_content,Tue Dec 19 2023 13:39:33 GMT-0700 (Mountain St...,Chrome Generic,Desktop,MacOSX,English,NaN,Health and Safety,Art Activities,folder,NaN,NaN,KoaliACL,NaN,NaN,Lesotho,2024,NaN
3,4,STC,open_module,Tue Dec 19 2023 13:44:15 GMT-0700 (Mountain St...,Chrome Generic,Desktop,MacOSX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KoaliACL,NaN,NaN,Lesotho,2024,NaN
4,5,Adding and Subtracting Integers,access_content,Wed Dec 20 2023 16:00:29 GMT-0700 (Mountain St...,Mobile Safari Generic,Mobile Phone,iOS,English,Fact Sheet,Math,Numbers and Counting,folder,NaN,NaN,KoaliACL,NaN,NaN,Lesotho,2024,NaN


### Step 5

In [ ]:
# STEP 5: Connect to MySQL
print("\n🔐 Connecting to database...")
db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url, connect_args={"ssl_ca": ssl_cert_filename})
print("✅ Connection established.")



🔐 Connecting to database...
✅ Connection established.


In [ ]:
# STEP 6: Create Table Dynamically
metadata = MetaData()
columns = [Column("id", BigInteger, primary_key=True)]
for col in df.columns[1:]:
    dtype = infer_sql_type(df[col])
    columns.append(Column(col, dtype))

table = Table(table_name, metadata, *columns,PrimaryKeyConstraint('id'))

try:
    print(f"\n⚙️ Creating table '{table_name}'...")
    metadata.drop_all(engine, [table], checkfirst=True)
    metadata.create_all(engine)
    print("✅ Table created.")

    print(f"⬆️ Uploading data to '{table_name}'...")
    df.to_sql(name=table_name, con=engine, if_exists="append", index=False)
    print("🎉 Upload successful!")
except Exception as e:
    print("❌ Upload failed:", e)


⚙️ Creating table 'shortresptranslatedb2r_rwanda'...
❌ Upload failed: (mysql.connector.errors.ConnectionTimeoutError) 2003: Can't connect to MySQL server on '10.177.128.15:3306' (Errno 110: Connection timed out)
(Background on this error at: https://sqlalche.me/e/20/dbapi)


# Checking mySQL Server

## Check what tables there are

This helps you check what tables are in the database

In [ ]:
from sqlalchemy import inspect

inspector = inspect(engine)
tables = inspector.get_table_names()
print("📋 Tables in database:", tables)


DBAPIError: (mysql.connector.errors.ConnectionTimeoutError) 2003: Can't connect to MySQL server on '10.177.128.15:3306' (Errno 110: Connection timed out)
(Background on this error at: https://sqlalche.me/e/20/dbapi)

## Display Tables

Helps you see the table to see if it looks right

In [ ]:
table_to_display = table_name

df = pd.read_sql_table(table_to_display, con=engine)
print(df.head())
print(df.tail())